# Tägliches Immowelt Webscraping

In diesem zweiten Notebook wird der Webscraping Code des ersten Notebooks, Web Analytics Project, angewendet und das Skript läuft täglich auf einem aws EC2 Server und speichert die gescrapten Daten je in eine neue Datei. 

## Inhaltsverzeichnis

1. [Installationen](#1-installationen)
2. [URL Untersuchung](#2-url-untersuchung)
3. [Webscraping](#3-webscraping)

## 1. Installationen

In [6]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import lxml
from lxml import html
from lxml import etree
import numpy as np
import pandas as pd
import requests
import re
import csv
from csv import writer

from scipy import stats
from datetime import date


## 2. URL Untersuchung

In [7]:
#1. Daten die abgefragt werden dedfinieren
Ort = ["berlin", "frankfurt-am-main", "hamburg", "koeln", "leipzig", "muenchen", "stuttgart"]
Umkreis = 50
Objekt = ["haeuser"]
Seite = list(range(50))

In [8]:
#2. URL aufteilen
Website = "https://www.immowelt.de/liste/"
Slash = "/"
Snippet1 = "/kaufen?d=true&efs=NEW_BUILDING_PROJECT&efs=JUDICIAL_SALE&r="
Snippet2 = "&sd=DESC&sf=RELEVANCE&sp="

## 3. Webscraping

In [9]:
#UPDATE SCHLEIFE

#12. Das ganze Automatisiert
#funktioniert alles Perfekt mit Fläche und Räume auch wenn manchmal nichts eingetragen ist
# wenn nichts eingetragen ist, ist der Eintrag leer
#csv Datei wird ausgegeben in dem Ordner wo es gespeichert ist
#es fehlt nur noch die maximale Seitenanzahl vom Button. Den auslesen und dann in die Schleifen einbinden
#weil bisher nur händische Eingabe der Seitenanzahl. Die Seitenanzahl ist mal auf zwei Seiten pro Suche 
#hier eingestellt
#benötigt etwas mehr als 15 Minuten


#ab hier bis zum nächsten hier ist 11.2 vorgeschaltet
AnzahlElementebyID=0

a = [[0 for j in range(len(Ort)*len(Seite)+10)] for i in range(len(Ort)*len(Seite)+10)]
for i in range(len(Ort)):
    for j in range(len(Seite)):
        a[i][j]= Website + Ort[i] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[j]+1)
        page = requests.get(a[i][j])
        tree = html.fromstring(page.content)
        id = tree.xpath(".//a/@id")
        
        AnzahlElementebyID=AnzahlElementebyID+len(id)
#hier 


#dann die Ausgabe definieren alles wie oben (Punkt 1. bis 11.) bis darauf, 
#dass alles in der Matrix b gespeichert wird 
#die Matrix b hat als Länge der Zeilen die Gesamtanzahl der Objekte über alle Seiten (AnzahlElementebyID)




#[[0 for j in range(Anzahl Spalten)] for i in range(Anzahl Zeilen)]; 
#a[i][j]=a[Zeile][Spalte]=AnzahlElementebyIDert in Zelle der Zeile i und Spalte j
b = [[0 for s in range(15)] for r in range(len(Ort)*AnzahlElementebyID)]
a = [[0 for j in range(len(Ort)*len(Seite)+10)] for i in range(len(Ort)*len(Seite)+10)]
laenge = 0
for i in range(len(Ort)):    
    for j in range(len(Seite)):
        a[i][j]= Website + Ort[i] + Slash + Objekt[0] +Snippet1 + str(Umkreis) + Snippet2 + str(Seite[j]+1)
        page = requests.get(a[i][j])
        tree = html.fromstring(page.content)
        id = tree.xpath(".//a/@id")
        prices = tree.xpath('//div[@data-test="price"]/text()')
        MaxOrt = tree.xpath('//div[@class="estateFacts-f11b0"]/div[1]/span/text()')
        for e in range(len(MaxOrt)):
            MaxOrt[e]=MaxOrt[e][MaxOrt[e].index(""): MaxOrt[e].index("|")-1] 
        
        rooms = [[0 for s in range(1)] for r in range(len(id))]
        area= [[0 for s in range(1)] for r in range(len(id))]
        
        for e in range(len(id)):
            rooms[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[3]/text()'% id[e])
            area[e][0]= tree.xpath('//a[@id="%s"]/div[2]/div[1]/div[1]/div[2]/text()'% id[e])
            
        
        
        
        for r in range(len(id)):
             b[laenge+r][0]= id[r] 
             b[laenge+r][1]= MaxOrt[r]     
             b[laenge+r][2]= prices[r]      
             b[laenge+r][3]= Ort[i]
             b[laenge+r][4]=Umkreis
             b[laenge+r][5]= area[r][0]
             b[laenge+r][6]= rooms[r][0]
             
             
           
             
        #laenge ist die Aktuelle Anzahl Häusern in der For Schleife 
        # for r in range(len(id)): für die erste Seite fängt es bei b[0+r][Beliebiger Platzhalter] an dann 
        # die zweite Seite ist die Häuseranzahl der ersten Seite laenge=0+len(id) also z.B laenge=20
        # dann fängt für die zweite Seite die b Matrix bei b[20+r][beliebiger Platzhalter] an und geht für
        # r = len(id) also die Anzahl der Häuser ab und addiert diese auf den letzten Wert der vorherigen Seite 
        # siehe  b[laenge+r][Beliebiger Platzhalter]  
        laenge = laenge + len(id) 
        
              
#Die Daten von der Einzelseite des Objekts scrapen

haus_url= [[0 for j in range(100000)] for i in range(AnzahlElementebyID+100)]
website = "https://www.immowelt.de/expose/"

for i in range(AnzahlElementebyID):
    haus_url[0][i] = website + str(b[i][0])
    
#for i in range(len(AnzahlElementebyID)):
    #print(haus_url[0][i])
    
for i in range(AnzahlElementebyID):    
    expose = requests.get(haus_url[0][i])
    ex_tree = html.fromstring(expose.content) 
    
    Grundstücksfläche= ex_tree.xpath(".//p[text()='Grundstücksfläche']")
    Kategorie= ex_tree.xpath(".//p[text()='Kategorie']")
    Etagen= ex_tree.xpath(".//p[text()='Geschosse']")
    Baujahr= ex_tree.xpath(".//p[text()='Baujahr']")
    Effizienzklasse= ex_tree.xpath(".//p[text()='Effizienzklasse']")
    Energieträger= ex_tree.xpath(".//p[text()='Energieträger']")
    Heizungsart= ex_tree.xpath(".//p[text()='Heizungsart']")
    
     
    b[i][7]= ex_tree.xpath('//div[@class="flex ng-star-inserted"]/div[3]/span/text()') 
        
    
    if (len(Kategorie)) >= 1:  
        b[i][8] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Kategorie"])]/p[2]/text()')
    else: 
        b[i][8]= "n.a" 
    
    if (len(Etagen)) >= 1:  
        b[i][9] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Geschosse"])]/p[2]/text()')
    else:
        b[i][9]= "n.a" 

    if (len(Baujahr)) >= 1:  
        b[i][10]= ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Baujahr"])]/p[2]/text()')
    else:
        b[i][10]= "n.a" 
   
    if (len(Effizienzklasse)) >= 1:  
        b[i][11] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Effizienzklasse"])]/p[2]/text()')
    else: 
        b[i][11]= "n.a" 
   
    if (len(Energieträger)) >= 1:  
        b[i][12] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Energieträger"])]/p[2]/text()')
    else:
        b[i][12]= "n.a"
    
    if (len(Heizungsart)) >= 1:  
        b[i][13] = ex_tree.xpath('//sd-cell-col[(@class="cell__col") and (.//p[text()="Heizungsart"])]/p[2]/text()')
    else:
      b[i][13]= "n.a"  
   
#jetzt noch in CSV Datei ausgeben:     

data = {
    'ID': [],
    'Ort': [],
    'Umkreis': [],
    'MaxOrt': [],
    'Preis': [],
    'Fläche': [],
    'Zimmer': [],
    'Grundstücksfläche': [],
    'Kategorie': [],
    'Etagen': [],
    'Baujahr': [],
    'Effizienzklasse': [],
    'Energieträger': [],
    'Heizungsart': [],
    'Stand':[],
}

for q in range(AnzahlElementebyID):
    data['ID'].append(b[q][0])
    data['Ort'].append(b[q][3])
    data['Umkreis'].append(b[q][4])
    data['MaxOrt'].append(b[q][1])
    data['Preis'].append(b[q][2])
    data['Fläche'].append(b[q][5])
    data['Zimmer'].append(b[q][6])
    data['Grundstücksfläche'].append(b[q][7])
    data['Kategorie'].append(b[q][8])
    data['Etagen'].append(b[q][9])
    data['Baujahr'].append(b[q][10])
    data['Effizienzklasse'].append(b[q][11])
    data['Energieträger'].append(b[q][12]) 
    data['Heizungsart'].append(b[q][13]) 
    data['Stand'].append(date.today())
      
     
     
     
     
df=pd.DataFrame(data, columns=['ID','Ort','Umkreis','MaxOrt','Preis','Fläche','Zimmer','Grundstücksfläche','Kategorie','Etagen','Baujahr','Effizienzklasse','Energieträger','Heizungsart','Stand']) 
df.to_csv(str(date.today()) +'_Immobilien.csv')